Source: https://www.kaggle.com/datasets/simranjain17/insurance

In [21]:
import pandas as pd
import sklearn.model_selection
import statsmodels.formula.api as smf
import numpy as np

salaries = pd.read_csv('insurance.csv')

data = salaries

smoker_map = {'yes': 1, 'no': 0}
data['smoker'] = data['smoker'].apply(lambda x: smoker_map[x])

data = pd.get_dummies(data, columns=['sex', 'region'])

# set dependent variable as first col for correlation matrix to show good results
cols = ['charges'] + [col for col in data if col != 'charges']
data = data[cols]

# print(data.info())

corr = data.corr()

X_train, X_test = sklearn.model_selection.train_test_split(data, test_size=0.2)
model = smf.ols(formula='charges ~ smoker + age + bmi', data=X_train).fit()
adj_R2 = model.rsquared

print(adj_R2)

0.7620411850770838
